In [63]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [45]:
dt = "2021-01"
df = pd.read_parquet(f"../resources/inputs/fhv_tripdata_{dt}.parquet")

In [46]:
df.shape

(1154112, 7)

In [53]:
df.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
duration                         float64
dtype: object

In [47]:
df.isnull().mean()

dispatching_base_num      0.000000
pickup_datetime           0.000000
dropOff_datetime          0.000000
PUlocationID              0.830307
DOlocationID              0.140558
SR_Flag                   1.000000
Affiliated_base_number    0.000767
dtype: float64

In [48]:
df["duration"] = (df["dropOff_datetime"] - df["pickup_datetime"]).dt.total_seconds() / 60

In [49]:
df["duration"].mean()

19.1672240937939

In [50]:
df = df[df["duration"].between(0, 60)]

In [51]:
df.shape

(1124850, 8)

In [52]:
df.isnull().mean()

dispatching_base_num      0.000000
pickup_datetime           0.000000
dropOff_datetime          0.000000
PUlocationID              0.831850
DOlocationID              0.132364
SR_Flag                   1.000000
Affiliated_base_number    0.000687
duration                  0.000000
dtype: float64

In [55]:
df = df.fillna({"PUlocationID": -1, "DOlocationID": -1})

In [57]:
categorical_feat = ["PUlocationID", "DOlocationID"]
y_name = "duration"

df[categorical_feat] = df[categorical_feat].astype(int).astype(str)

In [61]:
train_dicts = df[categorical_feat].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df[y_name].copy()

In [62]:
X_train.shape

(1124850, 525)

In [67]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.62970406166494

---